In [2]:
import pyspark
from pyspark.sql import SparkSession
import os

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
NESSIE_URI = os.getenv("NESSIE_URI")


def create_spark_session(appname):
    master = "spark://spark-master:7077"

    conf = (
        pyspark.SparkConf()
        .setAppName(appname)
        .set("spark.master", master)

        # EXTENSÕES ICEBERG + NESSIE
        .set(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )

        # REGISTRO DO CATÁLOGO NESSIE
        .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.nessie.type", "nessie")
        .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
        .set("spark.sql.catalog.nessie.ref", "main")
        .set("spark.sql.catalog.nessie.authentication.type", "NONE")
        .set("spark.sql.catalog.nessie.cache-enabled", "false")
        .set("spark.sql.catalog.nessie.warehouse", "s3a://lakehouse/")

        # CONFIG S3 -> ICEBERG
        .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
        .set("spark.sql.catalog.nessie.s3.endpoint", S3_ENDPOINT)

        # CONFIG HADOOP S3A
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
        .set("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
        .set("spark.hadoop.fs.s3a.path.style.access", "true")
        .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .set(
                "spark.hadoop.fs.s3a.aws.credentials.provider",
                "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider"
            )
        .set("spark.hadoop.fs.defaultFS", "s3a://lakehouse")

        # RECURSOS
        .set("spark.executor.memory", "1g")
        .set("spark.executor.cores", "1")
        .set("spark.driver.memory", "1g")
        .set("spark.executor.instances", "1")
    )

    spark = SparkSession.builder.config(conf=conf).getOrCreate()
    return spark



In [17]:
spark.stop()

In [19]:
spark = create_spark_session("bronze")

In [3]:
spark

In [14]:
from pyspark.sql import functions as F
from datetime import datetime
from uuid import uuid4
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


class Bronze:
    """
    Classe responsável pela camada Bronze do Lakehouse Tibia.

    A classe realiza:
    - Configuração do catálogo Nessie + Iceberg.
    - Criação dos namespaces e tabelas caso não existam.
    - Leitura dos arquivos CSV da camada Landing.
    - Padronização dos dados.
    - Registro de metadados operacionais.
    - Escrita incremental via Iceberg (append).

    Parameters
    ----------
    spark : SparkSession
        Sessão Spark ativa criada no job.
    date_str : str | None
        Data fornecida via CLI no formato 'YYYY-MM-DD'.
        Caso None, utiliza a data atual para construir o path da camada Landing.
    """

    def __init__(self, spark, date_str):
        self.spark = spark
        self.date_str = date_str

    # ============================================================
    #   MÉTODO: VOCATION
    # ============================================================
    def vocation(self):
        """
        Executa o job Bronze para dados de vocação (vocation).

        1. Configura catálogo Iceberg + warehouse.
        2. Cria namespace + tabela caso não existam.
        3. Lê arquivos CSV da camada landing.
        4. Valida colunas obrigatórias.
        5. Normaliza nomes e tipos de colunas.
        6. Gera batch_id e colunas de auditoria.
        7. Deduplica por (name, world).
        8. Insere incrementalmente na tabela Bronze.

        Tabela criada: nessie.bronze.vocation
        """

        # Namespace
        self.spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.bronze")

        # Criação da tabela Iceberg
        self.spark.sql("""
        CREATE TABLE IF NOT EXISTS nessie.bronze.vocation (
            name STRING,
            vocation STRING,
            level INT,
            world STRING,
            experience LONG,
            world_type STRING,
            ingestion_time TIMESTAMP,
            ingestion_date DATE,
            source_system STRING,
            batch_id STRING
        )
        USING iceberg
        PARTITIONED BY (world, ingestion_date)
        TBLPROPERTIES (
            'format-version' = '2',
            'write.format.default' = 'parquet',
            'write.metadata.compression' = 'gzip',
            'write.delete.mode' = 'merge-on-read'
        )
        """)

        # Define path da landing com base na data
        today_date = datetime.strptime(self.date_str, "%Y-%m-%d") if self.date_str else datetime.today()
        partition = f"year={today_date.year}/month={today_date.month}/day={today_date.day}"

        path = f"s3a://lakehouse/landing/{partition}/experience/"
        logging.info(f"Lendo dados de: {path}")

        # Lê arquivo CSV
        df_raw = self.spark.read.csv(path, header=True)

        # Validação de colunas esperadas
        colunas_esperadas = {"Rank", "Name", "Vocation", "World", "Level", "Points", "WorldType"}
        colunas_faltando = colunas_esperadas - set(df_raw.columns)

        if colunas_faltando:
            logging.error(f"Colunas ausentes no CSV: {colunas_faltando}")
            return

        # Gera batch_id para auditoria
        batch_id = str(uuid4())
        logging.info(f"Gerando batch_id: {batch_id}")

        df_raw.printSchema()

        # Normalização e padronização
        df_bronze = (
            df_raw.drop("Rank")
            .withColumnRenamed("Name", "name")
            .withColumnRenamed("Vocation", "vocation")
            .withColumnRenamed("Level", "level")
            .withColumnRenamed("World", "world")
            .withColumnRenamed("Points", "experience")
            .withColumnRenamed("WorldType", "world_type")
            .withColumn("ingestion_time", F.current_timestamp())
            .withColumn("ingestion_date", F.current_date())
            .withColumn("source_system", F.lit("highscore_tibia_page"))
            .withColumn("batch_id", F.lit(batch_id))
            .withColumn("experience", F.regexp_replace("experience", ",", "").cast("long"))
            .withColumn("level", F.col("level").cast("int"))
            .withColumn("vocation", F.trim(F.lower("vocation")))
            .withColumn("world", F.trim(F.lower("world")))
            .dropDuplicates(["name", "world"])
        )

        # Compressão padrão Parquet
        self.spark.conf.set("spark.sql.parquet.compression.codec", "snappy")

        record_count = df_bronze.count()

        if record_count > 0:
            logging.info(f"Inserindo {record_count} registros na Bronze com batch_id {batch_id}...")
            df_bronze.writeTo("nessie.bronze.vocation").append()
        else:
            logging.warning("Nenhum registro encontrado para gravar na Bronze.")

In [15]:
bronze = Bronze(spark, "2025-12-25")

In [16]:
bronze.vocation()

2025-12-25 23:57:23,117 - INFO - Lendo dados de: s3a://lakehouse/landing/year=2025/month=12/day=25/experience/
2025-12-25 23:57:23,797 - INFO - Gerando batch_id: e852ac20-c5d1-4d23-9c31-ae84045a88af


root
 |-- Rank: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Vocation: string (nullable = true)
 |-- World: string (nullable = true)
 |-- Level: string (nullable = true)
 |-- Points: string (nullable = true)
 |-- WorldType: string (nullable = true)



2025-12-25 23:57:25,394 - INFO - Inserindo 500 registros na Bronze com batch_id e852ac20-c5d1-4d23-9c31-ae84045a88af...


In [17]:
spark.stop()

In [20]:
df = spark.read.table("nessie.bronze.vocation")
df.count().show(5

+-------------+-----------+-----+-------+-----------+--------------+--------------------+--------------+--------------------+--------------------+
|         name|   vocation|level|  world| experience|    world_type|      ingestion_time|ingestion_date|       source_system|            batch_id|
+-------------+-----------+-----+-------+-----------+--------------+--------------------+--------------+--------------------+--------------------+
|   Appov Boss|elder druid| 1017|eclipta|17452496195|Retro Open PvP|2025-12-25 23:47:...|    2025-12-25|highscore_tibia_page|f782c035-7bfc-4a0...|
|        Aztev|elder druid| 1025|eclipta|17893581139|Retro Open PvP|2025-12-25 23:47:...|    2025-12-25|highscore_tibia_page|f782c035-7bfc-4a0...|
|   Bre no zin|elder druid|  874|eclipta|11084969190|Retro Open PvP|2025-12-25 23:47:...|    2025-12-25|highscore_tibia_page|f782c035-7bfc-4a0...|
|Brunon Brunon|elder druid| 1168|eclipta|26432758562|Retro Open PvP|2025-12-25 23:47:...|    2025-12-25|highscore_tibi

In [21]:
df.count()

1000

In [22]:
spark.stop()

In [2]:
from src.jobs.utils.silver import Silver

In [6]:
spark = create_spark_session(f"silver_vocation")

# Instancia a camada Silver
silver = Silver(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 12:30:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
silver.vocation()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
2025-12-27 12:31:37,998 - INFO - Tabela Silver inicializada com sucesso.
2025-12-27 12:31:48,731 - INFO - View temporária `vocation_updates` criada com sucesso.
2025-12-27 12:32:45,762 - INFO - MERGE INTO finalizado com sucesso!             
2025-12-27 12:32:59,865 - INFO - Total de registros na Silver: 100054 + 2) / 56]
2025-12-27 12:32:59,867 - INFO - Registros atuais (is_current = true): 1000


In [8]:
spark.stop()

In [2]:
from src.jobs.utils.bronze import Bronze

In [3]:
spark = create_spark_session("bronze")
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 13:03:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
bronze = Bronze(spark, "2025-12-27")

In [5]:
bronze.skills()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
2025-12-27 13:04:12,194 - INFO - Lendo dados de: s3a://lakehouse/landing/year=2025/month=12/day=27/skills/
25/12/27 13:04:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
2025-12-27 13:04:20,092 - INFO - Gerando batch_id: bb4d5c13-1029-46e2-afa4-57d98353ccdc
2025-12-27 13:04:28,045 - INFO - Inserindo 1100 registros na Bronze com batch_id bb4d5c13-1029-46e2-afa4-57d98353ccdc...


In [9]:
(spark.read.table("nessie.bronze.skills").show())

+---------------+-------------+-------+-----+-----------+-----------------+--------------------+--------------+--------------------+--------------------+
|           name|     vocation|  world|level|skill_level|         category|      ingestion_time|ingestion_date|       source_system|            batch_id|
+---------------+-------------+-------+-----+-----------+-----------------+--------------------+--------------+--------------------+--------------------+
|        Anxious|royal paladin|refugia|  629|        136|Distance Fighting|2025-12-27 13:04:...|    2025-12-27|highscore_tibia_page|bb4d5c13-1029-46e...|
|          Arbia|royal paladin|refugia|  379|        134|Distance Fighting|2025-12-27 13:04:...|    2025-12-27|highscore_tibia_page|bb4d5c13-1029-46e...|
|         Aryenn|royal paladin|refugia|  534|        133|Distance Fighting|2025-12-27 13:04:...|    2025-12-27|highscore_tibia_page|bb4d5c13-1029-46e...|
|       Asymides|royal paladin|refugia| 1076|        138|Distance Fighting|2

In [10]:
(
    spark.read.table("nessie.bronze.skills")
    .groupBy("category")
    .count()
    .orderBy("count", ascending=False)
    .show()
)


+-----------------+-----+
|         category|count|
+-----------------+-----+
|Distance Fighting| 1000|
|      Magic Level|   50|
|   Sword Fighting|   50|
+-----------------+-----+



In [13]:
spark.stop()

In [12]:
from src.jobs.utils.silver import Silver

In [14]:
spark = create_spark_session("silver_skills")

# Instancia a camada Silver
silver = Silver(spark)

In [15]:
silver.skills()

2025-12-27 13:10:32,112 - INFO - Tabela Silver 'skills' inicializada com sucesso.
2025-12-27 13:10:36,508 - INFO - View temporária 'skills_updates' criada com sucesso.
2025-12-27 13:11:31,408 - INFO - MERGE INTO concluído com sucesso!              
2025-12-27 13:11:42,622 - INFO - Total de registros na Silver: 1100             
2025-12-27 13:11:42,624 - INFO - Registros atuais (is_current = true): 1100


In [16]:
(spark.read.table("nessie.silver.skills").show())

+--------------------+-------------+-------+-----+-----------+-----------------+--------------------+--------------------+--------+----------+
|                name|     vocation|  world|level|skill_level|         category|      ingestion_time|          start_date|end_date|is_current|
+--------------------+-------------+-------+-----+-----------+-----------------+--------------------+--------------------+--------+----------+
|   Artista Economico|royal paladin| inabra| 1426|        134|Distance Fighting|2025-12-27 13:04:...|2025-12-27 13:10:...|    NULL|      true|
|           Mondarius|royal paladin| inabra| 1258|        139|Distance Fighting|2025-12-27 13:04:...|2025-12-27 13:10:...|    NULL|      true|
|           Cros Bolt|royal paladin| inabra| 1182|        137|Distance Fighting|2025-12-27 13:04:...|2025-12-27 13:10:...|    NULL|      true|
|Ellmagnifinho Dan...|royal paladin| inabra| 2188|        140|Distance Fighting|2025-12-27 13:04:...|2025-12-27 13:10:...|    NULL|      true|

In [4]:
spark.stop()

In [5]:
from src.jobs.utils.bronze import Bronze
spark = create_spark_session("bronze")
bronze = Bronze(spark, "2025-12-27")

In [6]:
bronze.extra()

2025-12-27 13:25:16,354 - INFO - Lendo dados de: s3a://lakehouse/landing/year=2025/month=12/day=27/extra/
2025-12-27 13:25:39,017 - INFO - Inserindo 100 registros na Bronze Extra com batch_id 9967cc6d-6436-4936-880d-c4434df477bd...
2025-12-27 13:25:48,964 - INFO - Carga concluída com sucesso.                   


In [7]:
spark.stop()

In [8]:
from src.jobs.utils.silver import Silver
spark = create_spark_session("silver_extra")
silver = Silver(spark)
silver.extra()

2025-12-27 13:26:26,147 - INFO - Tabela Silver 'extra' inicializada com sucesso.
2025-12-27 13:26:38,024 - INFO - View temporária 'extra_updates' criada com sucesso.
2025-12-27 13:27:00,759 - INFO - MERGE INTO Silver Extra finalizado com sucesso!
2025-12-27 13:27:04,382 - INFO - Total de registros na Silver Extra: 100        
2025-12-27 13:27:04,383 - INFO - Registros atuais (is_current = true): 100


In [3]:
%run src/jobs/bronze_job.py skills --date 2025-12-27


25/12/27 13:37:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2025-12-27 13:37:09,570 - INFO - Lendo dados de: s3a://lakehouse/landing/year=2025/month=12/day=27/skills/
2025-12-27 13:37:15,239 - INFO - Gerando batch_id: 07468fb7-3c14-4739-8f6d-3a2294ce91b9
2025-12-27 13:37:25,005 - INFO - Inserindo 1100 registros na Bronze com batch_id 07468fb7-3c14-4739-8f6d-3a2294ce91b9...
2025-12-27 13:37:38,586 - INFO - Camada bronze finalizada com sucesso.          


In [4]:
%run src/jobs/bronze_job.py extra --date 2025-12-27


25/12/27 13:37:46 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2025-12-27 13:37:46,564 - INFO - Lendo dados de: s3a://lakehouse/landing/year=2025/month=12/day=27/extra/
2025-12-27 13:37:49,179 - INFO - Inserindo 100 registros na Bronze Extra com batch_id de57e20a-fc1f-46c1-982f-20ad3691a372...
2025-12-27 13:37:56,387 - INFO - Carga concluída com sucesso.                   
2025-12-27 13:37:56,391 - INFO - Camada bronze finalizada com sucesso.


In [2]:
%run src/jobs/silver_job.py vocation


25/12/27 13:51:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2025-12-27 13:51:50,106 - INFO - Tabela Silver inicializada com sucesso.
2025-12-27 13:51:54,312 - INFO - Processando batch_id: e852ac20-c5d1-4d23-9c31-ae84045a88af
2025-12-27 13:51:55,113 - INFO - View temporária `vocation_updates` criada com sucesso.
2025-12-27 13:52:40,510 - INFO - MERGE INTO finalizado com sucesso!             
2025-12-27 13:52:45,658 - INFO - Total de registros na Silver: 1000             
2025-12-27 13:52:45,664 - INFO - Registros atuais (is_current = true): 1000
2025-12-27 13:52:45,666 - INFO - Camada silver finalizada com sucesso.


In [11]:
%run src/jobs/silver_job.py extra

2025-12-27 14:19:21,017 - INFO - Tabela Silver 'extra' inicializada com sucesso.
2025-12-27 14:19:22,826 - INFO - Processando batch_id: de57e20a-fc1f-46c1-982f-20ad3691a372
2025-12-27 14:19:23,126 - INFO - View temporária 'extra_updates' criada com sucesso.
2025-12-27 14:19:38,818 - INFO - MERGE INTO Silver Extra finalizado com sucesso.
2025-12-27 14:19:41,310 - INFO - Total de registros: 100                        
2025-12-27 14:19:41,311 - INFO - Registros atuais (is_current = true): 100
2025-12-27 14:19:41,312 - INFO - Camada silver finalizada com sucesso.


In [10]:
spark.sql("""
SELECT
  name,
  world,
  category,
  title,
  COUNT(*) AS qtd
FROM nessie.silver.extra
WHERE is_current = true
GROUP BY
  name,
  world,
  category,
  title
HAVING COUNT(*) > 1
""").show()


+--------------------+--------+-----------+-----+---+
|                name|   world|   category|title|qtd|
+--------------------+--------+-----------+-----+---+
|                Teus|descubra|Drome Score| NULL|  2|
|             Daso II|  antica|Drome Score| NULL|  2|
|      Blaina Siwynia|  antica|Boss Points| NULL|  2|
| Benedicto De Amorim|  inabra|Boss Points| NULL|  2|
|              Goraca|    bona|Drome Score| NULL|  2|
|             Kilseno|  secura|Boss Points| NULL|  2|
|     Werewolf Shorti|  secura|Boss Points| NULL|  2|
|          Ravenovsky|  secura|Boss Points| NULL|  2|
|               Acidd|descubra|Boss Points| NULL|  2|
|     Incredible Toby|  secura|Boss Points| NULL|  2|
|         Ale Devasso|collabra|Drome Score| NULL|  2|
|               Celso|   nevia|Boss Points| NULL|  2|
|              Szimix| celesta|Drome Score| NULL|  2|
|             Szmuggy|  thyria|Drome Score| NULL|  2|
|            Exorinho|  thyria|Drome Score| NULL|  2|
|             Al Meow|  thyr

In [4]:
spark = create_spark_session("silver")


In [12]:
spark.sql("""SELECT
  name, world, category, title,
  COUNT(*) AS qtd
FROM nessie.silver.extra
WHERE is_current = true
GROUP BY name, world, category, title
HAVING COUNT(*) > 1;
""").show()

+----+-----+--------+-----+---+
|name|world|category|title|qtd|
+----+-----+--------+-----+---+
+----+-----+--------+-----+---+



In [7]:
spark.sql(""" DROP TABLE nessie.silver.skills""")

DataFrame[]

In [10]:
%run src/jobs/silver_job.py skills

2025-12-27 14:36:18,662 - INFO - Tabela Silver 'skills' inicializada com sucesso.
2025-12-27 14:36:20,669 - INFO - Processando batch_id: 07468fb7-3c14-4739-8f6d-3a2294ce91b9
2025-12-27 14:36:21,044 - INFO - View temporária 'skills_updates' criada com sucesso.
2025-12-27 14:37:33,410 - INFO - MERGE INTO concluído com sucesso!              
2025-12-27 14:37:39,095 - INFO - Total de registros na Silver: 1100             
2025-12-27 14:37:39,101 - INFO - Registros atuais (is_current = true): 1100
2025-12-27 14:37:39,103 - INFO - Camada silver finalizada com sucesso.


In [9]:
spark.sql("""SELECT name, world, category, COUNT(*)
FROM nessie.silver.skills
WHERE is_current = true
GROUP BY name, world, category
HAVING COUNT(*) > 1;""").show()


+----+-----+--------+--------+
|name|world|category|count(1)|
+----+-----+--------+--------+
+----+-----+--------+--------+



In [17]:
spark.sql(""" DROP TABLE nessie.silver.vocation""")

DataFrame[]

In [5]:
%run src/jobs/silver_job.py vocation

2025-12-27 14:32:54,379 - INFO - Tabela Silver inicializada com sucesso.
2025-12-27 14:32:57,165 - INFO - Processando batch_id: e852ac20-c5d1-4d23-9c31-ae84045a88af
2025-12-27 14:32:57,497 - INFO - View temporária `vocation_updates` criada com sucesso.
2025-12-27 14:33:34,552 - INFO - MERGE INTO finalizado com sucesso!             
2025-12-27 14:33:38,140 - INFO - Total de registros na Silver: 500              
2025-12-27 14:33:38,142 - INFO - Registros atuais (is_current = true): 500
2025-12-27 14:33:38,144 - INFO - Camada silver finalizada com sucesso.


In [6]:
spark.sql("""SELECT name, world, COUNT(*)
FROM nessie.silver.vocation
WHERE is_current = true
GROUP BY name, world
HAVING COUNT(*) > 1;""").show()


+----+-----+--------+
|name|world|count(1)|
+----+-----+--------+
+----+-----+--------+



In [11]:
spark.stop()